# Getting started with the CEDA STAC API

This notebook outlines how to use the CEDA STAC API's transaction endpoint, using a the client credentials flow. 


## 1. Imports & Setup
First we import `httpx`, `yaml`, and `json` libraries.
And load the config and test collection and item.

In [12]:
import httpx
import yaml
import json
import os
from httpx_auth import OAuth2ClientCredentials

with open(".transactions-settings.yml") as reader:
    conf = yaml.safe_load(reader)

api_root_url = conf.get("API_ROOT_URL", "http://localhost:8000")

headers = {
    "client_id": conf.get("CLIENT_ID"),
    "client_secret": conf.get("CLIENT_KEY")
}
# client = httpx.Client(headers=headers)
auth=OAuth2ClientCredentials(
    conf.get("TOKEN_URL"),
    client_id=conf.get("CLIENT_ID"),
    client_secret=conf.get("CLIENT_KEY")
)
client = httpx.Client(auth=auth)

with open("data/collection.json") as collection_file, open("data/item.json") as item_file:
    collection = json.load(collection_file)
    item = json.load(item_file)

## 2. Collection transaction endpoints

### a. Adding a collection
Now we can define the collections endpoint and post our test collection to it.

In [19]:
collections_url = os.path.join(api_root_url, "collections")
r = client.post(collections_url, json=collection)
r

{'type': 'Collection', 'stac_version': '1.0.0', 'stac_extensions': [], 'id': 'cmip6', 'extent': {'temporal': {'interval': None}, 'spatial': {'bbox': None}}, 'summaries': {'uri': 'cmip6', 'mip_era': ['CMIP6'], 'activity_id': ['C4MIP', 'CMIP'], 'institution_id': ['MOHC'], 'source_id': ['UKESM1-0-LL'], 'table_id': ['Amon'], 'var_id': ['tas'], 'version': ['v20190502', 'v20190806', 'v20191210']}, 'assets': {}}


'{"code":"ConflictError","description":"Collection cmip6 already exists"}'

### b. Updating a collection
An existing collection can be update at the collections endpoint.

In [25]:
collection_url = os.path.join(collections_url, collection.get("id", "test_collection"))
updated_collection = collection | {"summaries": collection["summaries"] | {"another_facet": ["Hello world"]}}
print(updated_collection)
# Currently using the collections endpoint this will change once 
# https://github.com/stac-utils/stac-fastapi/pull/631 is accepted
r = client.put(collection_url, json=updated_collection)
r

{'type': 'Collection', 'stac_version': '1.0.0', 'stac_extensions': [], 'id': 'cmip6', 'extent': {'temporal': {'interval': None}, 'spatial': {'bbox': None}}, 'summaries': {'uri': 'cmip6', 'mip_era': ['CMIP6'], 'activity_id': ['C4MIP', 'CMIP'], 'institution_id': ['MOHC'], 'source_id': ['UKESM1-0-LL'], 'table_id': ['Amon'], 'var_id': ['tas'], 'version': ['v20190502', 'v20190806', 'v20191210'], 'another_facet': ['Hello world']}, 'assets': {}}


<Response [500 Internal Server Error]>

### c. Removing a collection
The collection can also be removed. **Note** removing a collection will also remove all of it's items.

In [27]:
r = client.delete(collection_url)
r

<Response [500 Internal Server Error]>

## 3. Item transaction endpoints

### a. Adding an item
Now we can define the item endpoint and post our test item to it. **Note** The item's collection must exist before the item is created.

In [5]:
items_url = os.path.join(collection_url, "items")
r = client.post(collections_url, json=collection)

### b. Updating an item
An existing item can be update at the desired item's endpoint.

In [ ]:
item_url = os.path.join(items_url,  item.get("id", "test_item"))
updated_item = item | {"properties": {"another_facet": "Hello world"}}
r = client.put(item_url, data=updated_item)

### c. Removing an item
The item can also be removed.

In [ ]:
r = client.delete(item_url)